In [24]:
import requests
import time 
from bs4 import BeautifulSoup

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from requests.exceptions import RequestException
import re
import pandas as pd
res_users=[]
res_pingfen=[]
res_times=[]
res_comments=[]

def get_one_page(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.4 Safari/537.36"
    }
    try:
        response=requests.get(url,headers=headers)
        if(response.status_code==200):
            response.encoding='utf-8'
            return response.text
        return None
    except RequestException:
        print("error")
        return None 

def parse_one_page(html):
    soup=BeautifulSoup(html,'lxml')
    comment_lists=soup.text
    
    #获取一个页面的用户名
    user_ls='"nickname":"(.*?)"'
    user_pattern = re.compile(user_ls, re.S)
    user_lists = re.findall(user_pattern, str(comment_lists))
    for li in user_lists:
        #print(li)
        res_users.append(li)
     
    #获取一个页面的评分
    fen_ls = '"pointValue":(\d)'
    fen_pattern = re.compile(fen_ls, re.S)
    fen_lists = re.findall(fen_pattern, str(comment_lists))
    for li in fen_lists:
        #print(li)
        res_pingfen.append(li)
        
    #获取一个页面的时间"
    shijian_ls = '"updateTime":(\d+)'
    shijian_pattern = re.compile(shijian_ls, re.S)
    shijian_lists = re.findall(shijian_pattern, str(comment_lists))
    for li in shijian_lists:
        #print(li)
        res_times.append(li)
    
    #获取一个页面的评论"
    pinglun_ls = '"commentValue":"(.*?)"'
    pinglun_pattern = re.compile(pinglun_ls, re.S)
    pinglun_lists = re.findall(pinglun_pattern, str(comment_lists))
    for li in pinglun_lists:
        #print(li)
        res_comments.append(li)

def pachong(offset):
    url='https://app.market.xiaomi.com/apm/comment/list/428633?clientId=5e7b8d12c7cfd03253f6213f9430d040&co=CN&densityScaleFactor=3.0&deviceType=0&imei=c689938c2cb64ead30ce1ae2c77504de&la=zh&lo=CN&mac=5e7b8d12c7cfd03253f6213f9430d040&marketVersion=1914103&miuiBigVersionCode=5&miuiBigVersionName=V7&model=Mi-4c&os=V7.0.9.0.LXKCNCI&page='+str(offset)+'&pageConfigVersion=171&pageRef=index&resolution=1080*1920&ro=unknown&sdk=22&session=TeIf9malXpbi1TQK&webResVersion=736'
    html = get_one_page(url)
    parse_one_page(html)

def xiaomi(file_path):
    start_time = time.time() 
    pool = ThreadPool(5)
    pool.map(pachong,[i for i in range(1,200)])
    pool.close()
    pool.join()
    data=[res_users,res_pingfen,res_times,res_comments]
    result=pd.DataFrame(data)
    result=result.T
    result.columns=['用户名称','评分','时间','评论']
    result.to_csv(file_path+'小米_平安口袋银行.csv')   

if __name__=='__main__':
    file_path='C:\\Users\\pujing\\Desktop\\长亮科技\\'
    xiaomi(file_path)
    print("爬取完成！")

爬取完成！
